In [1]:
import matplotlib.image as img
import numpy as np
import pandas as pd 
from sklearn.neighbors import KNeighborsRegressor
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from scipy import stats

from PIL import Image

df = pd.read_csv('./data/fashion-mnist_train.csv')
display(df.shape)

x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df.drop('label', axis=1, inplace=False), 
                 df['label'], test_size=0.3, random_state=0)

## nomalization
scaler = MinMaxScaler()
scaler.fit(x_data_train)
x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)
del x_data_train, x_data_test

x_data_train_norm = x_data_train_norm.reshape((-1, 28, 28, 1))
x_data_test_norm = x_data_test_norm.reshape((-1, 28, 28, 1))

print(x_data_train_norm.shape)
print(x_data_test_norm.shape)
print(t_data_train.shape)
print(t_data_test.shape)


with tf.device('/device:GPU:1'):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))

    model.add(Flatten())
    model.add(Dropout(rate=0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    model.summary()

    # 학습실행
    model.compile(optimizer=Adam(learning_rate=1e-3),
                  loss='sparse_categorical_crossentropy',
                  metrics=['sparse_categorical_accuracy'])

    # model.fit(x_data_train_norm, t_data_train, epochs=5)
    history = model.fit(x_data_train_norm,
                        t_data_train,
                        epochs=100,
                        verbose=1,
                        batch_size=100,
                        validation_split=0.3)

(60000, 785)

(42000, 28, 28, 1)
(18000, 28, 28, 1)
(42000,)
(18000,)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0         
_________________________________________________________________


294/294 [==============================] - 4s 13ms/step - loss: 0.0819 - sparse_categorical_accuracy: 0.9688 - val_loss: 0.3057 - val_sparse_categorical_accuracy: 0.9123
Epoch 38/100
294/294 [==============================] - 4s 13ms/step - loss: 0.0824 - sparse_categorical_accuracy: 0.9689 - val_loss: 0.3226 - val_sparse_categorical_accuracy: 0.9148
Epoch 39/100
294/294 [==============================] - 4s 13ms/step - loss: 0.0821 - sparse_categorical_accuracy: 0.9690 - val_loss: 0.3220 - val_sparse_categorical_accuracy: 0.9131
Epoch 40/100
294/294 [==============================] - 4s 13ms/step - loss: 0.0757 - sparse_categorical_accuracy: 0.9713 - val_loss: 0.3170 - val_sparse_categorical_accuracy: 0.9152
Epoch 41/100
294/294 [==============================] - 4s 13ms/step - loss: 0.0759 - sparse_categorical_accuracy: 0.9716 - val_loss: 0.3184 - val_sparse_categorical_accuracy: 0.9129
Epoch 42/100
294/294 [==============================] - 4s 13ms/step - loss: 0.0695 - sparse_categ

Epoch 82/100
294/294 [==============================] - 4s 13ms/step - loss: 0.0421 - sparse_categorical_accuracy: 0.9850 - val_loss: 0.4292 - val_sparse_categorical_accuracy: 0.9155
Epoch 83/100
294/294 [==============================] - 4s 13ms/step - loss: 0.0380 - sparse_categorical_accuracy: 0.9866 - val_loss: 0.4446 - val_sparse_categorical_accuracy: 0.9134
Epoch 84/100
294/294 [==============================] - 4s 13ms/step - loss: 0.0454 - sparse_categorical_accuracy: 0.9844 - val_loss: 0.4336 - val_sparse_categorical_accuracy: 0.9133
Epoch 85/100
294/294 [==============================] - 4s 13ms/step - loss: 0.0436 - sparse_categorical_accuracy: 0.9843 - val_loss: 0.4259 - val_sparse_categorical_accuracy: 0.9098
Epoch 86/100
294/294 [==============================] - 4s 13ms/step - loss: 0.0377 - sparse_categorical_accuracy: 0.9863 - val_loss: 0.4502 - val_sparse_categorical_accuracy: 0.9114
Epoch 87/100
294/294 [==============================] - 4s 13ms/step - loss: 0.0419 -

In [4]:
result = np.argmax(model.predict(x_data_test_norm), axis=1)
print(classification_report(t_data_test, result))
model.evaluate(x_data_test_norm, t_data_test)

print(history.history.keys())
plt.plot(history.history['sparse_categorical_accuracy'], color='r')
plt.plot(history.history['val_sparse_categorical_accuracy'], color='b')
plt.show()
plt.plot(history.history['loss'], color='r')
plt.plot(history.history['val_loss'], color='b')
plt.show()

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Conv2D (defined at <ipython-input-2-2d83529b2318>:1) ]] [Op:__inference_predict_function_91425]

Function call stack:
predict_function
